In [1]:
# packages
import os
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report, confusion_matrix, precision_score, f1_score
from sklearn.model_selection import TimeSeriesSplit, KFold
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, ReLU, GlobalAveragePooling1D, Dense
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical


In [2]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/ms_wind_curtailment_prediction/lagged_curtailment_target_features.csv', sep = ';', index_col=0)

In [4]:
# convert index to datetime type
df.index = pd.to_datetime(df.index)
# drop wind speed to reduce multicollinearity
df.drop(['wind_speed_m/s', 'wind_speed_m/s_lag1', 'wind_speed_m/s_lag2'], inplace=True, axis=1)

In [5]:
# get desired df size
start_date = '2022-01-01'
end_date = '2023-12-31'
df = df.loc[start_date:end_date]

In [6]:
# impute, scale pipeline and smote (for class imbalance)
preprocessor = Pipeline([
    ('scaler', StandardScaler())
])

smote = SMOTE(random_state=13)

# define features X and target y
X = df.drop(['redispatch', 'level'], axis = 1)
y = df['redispatch']

In [7]:
# Define model architecture
def make_model(input_shape, num_classes):
    input_layer = Input(input_shape)

    conv1 = Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = BatchNormalization()(conv1)
    conv1 = ReLU()(conv1)

    conv2 = Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = ReLU()(conv2)

    conv3 = Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = BatchNormalization()(conv3)
    conv3 = ReLU()(conv3)

    gap = GlobalAveragePooling1D()(conv3)

    output_layer = Dense(num_classes, activation="softmax")(gap)

    return Model(inputs=input_layer, outputs=output_layer)

# Define callbacks
callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001),
    EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]

In [8]:
# Fit the pipeline on the entire dataset
X_scaled = preprocessor.fit_transform(X)

In [ ]:
# Define the number of splits and test size for time series cross-validation
n_splits = 50 #500
test_size = 48 #(48 - 12h with 15 min intervals)
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_size)

# Initialize lists to store evaluation metrics for each fold
train_f1_scores = []
train_precision_scores = []
test_f1_scores = []
test_precision_scores = []

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(tscv.split(X_scaled), 1):
    print(f"Training on fold {fold}/{n_splits}")

    # Get the data for this fold
    X_train_fold, X_test_fold = X_scaled[train_index], X_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # Apply SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train_fold, y_train_fold)

    # Reshape the input data to include the sequence length dimension
    X_train_balanced_reshaped = np.expand_dims(X_train_balanced, axis=-1)
    X_test_fold_reshaped = np.expand_dims(X_test_fold, axis=-1)

    # Define input shape based on reshaped input data
    input_shape = X_train_balanced_reshaped.shape[1:]
    num_classes = len(np.unique(y))

    # Create model instance
    model = make_model(input_shape, num_classes)

    # Compile the model
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

 # Train the model
    with tf.device('/device:GPU:0'):
      # Train the model
      history = model.fit(
          X_train_balanced, y_train_balanced,
          validation_data=(X_test_fold, y_test_fold),
          epochs=500,
          batch_size=32,
          callbacks=callbacks,
          verbose=1
      )

    # Predict on the train set
    y_pred_train = model.predict(X_train_balanced)
    y_pred_train_classes = np.argmax(y_pred_train, axis=1)

    # Calculate evaluation metrics for train set
    train_f1 = f1_score(y_train_balanced, y_pred_train_classes, average='macro')
    train_precision = precision_score(y_train_balanced, y_pred_train_classes, average='macro')

    train_f1_scores.append(train_f1)
    train_precision_scores.append(train_precision)

    # Predict on the test set
    y_pred_test = model.predict(X_test_fold)
    y_pred_test_classes = np.argmax(y_pred_test, axis=1)

    # Calculate evaluation metrics for test set
    test_f1 = f1_score(y_test_fold, y_pred_test_classes, average='macro')
    test_precision = precision_score(y_test_fold, y_pred_test_classes, average='macro')

    test_f1_scores.append(test_f1)
    test_precision_scores.append(test_precision)

# Print average scores across all folds
print("Average F1 score (train):", np.mean(train_f1_scores))
print("Average precision score (train):", np.mean(train_precision_scores))
print("Average F1 score (test):", np.mean(test_f1_scores))
print("Average precision score (test):", np.mean(test_precision_scores))

Training on fold 1/50
Epoch 1/500
3716/3716 [==============================] - 24s 5ms/step - loss: 0.5874 - accuracy: 0.6818 - val_loss: 0.0478 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/500
3716/3716 [==============================] - 19s 5ms/step - loss: 0.5482 - accuracy: 0.7162 - val_loss: 0.1267 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/500
3716/3716 [==============================] - 18s 5ms/step - loss: 0.5298 - accuracy: 0.7301 - val_loss: 0.2313 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/500
3716/3716 [==============================] - 19s 5ms/step - loss: 0.5148 - accuracy: 0.7422 - val_loss: 0.1636 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/500
3716/3716 [==============================] - 18s 5ms/step - loss: 0.5025 - accuracy: 0.7511 - val_loss: 0.1939 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/500
3716/3716 [==============================] - 19s 5ms/step - loss: 0.4932 - accuracy: 0.7564 - val_loss: 0.4321 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 7/500
3716/3716 [=